In [1]:
import numpy as np
import pandas as pd

In [3]:
split_pair_path = "../data/split_test/split_pair_test.txt"
train_data_path = "../data/split_test/train_set.txt"
test_data_path = "../data/split_test/test_tweets_unlabeled_dataframe_v1_4.txt"


train_data = pd.read_csv(train_data_path, sep='\t', header=None)
train_label = np.array(train_data.iloc[:,0]).astype('int')
test_data = pd.read_csv(test_data_path)
split_data = pd.read_csv(split_pair_path, sep='\t')

In [9]:
split_data

,true_sentence,candidate_sentence,candidate_label
0,Some people say that rappers don’t have feelin...,"DAYUM!! ..that's a deep ass quote! ""Love is gi...",8327
1,Some people say that rappers don’t have feelin...,"The fact is men will use whatever they have, b...",9048
2,Some people say that rappers don’t have feelin...,"Gotta keep it 100. All men ain't the same, jus...",763
3,Some people say that rappers don’t have feelin...,It has nothing to do with being loyal. If I do...,4936
4,Some people say that rappers don’t have feelin...,"""You're not an artist. You're an art thief! If...",9902
5,Some people say that rappers don’t have feelin...,It's not sexy for two women to fight over a du...,3729
6,Some people say that rappers don’t have feelin...,#whymencheat .....Women might be able to fake ...,6751
7,Some people say that rappers don’t have feelin...,"Women sometimes fake orgasms,, but ladies WATC...",3936
8,Some people say that rappers don’t have feelin...,Malignaggi went off on boxing after that fight...,886
9,Some people say that rappers don’t have feelin...,I like Gucci Mane in the strip club. I like Im...,2722


In [5]:
# from probability file, get top k candidates
probability_file_path = "../data/split_test/test_results_3.tsv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]


In [33]:
%%time
top = 20
new_top = 20
probability_threshold = 1
new_candidate_list = np.zeros((int(len(split_data)/top),1)).astype('int')
counter = 0
for i in range(0, len(split_data), top):
    new_train_data_single_split = split_data[i:(i+top)]
    new_train_label_single_split = np.array(new_train_data_single_split.candidate_label)
    probability_single_split = probability_file[i:(i+top)]
    sorted_index = np.array(np.argsort(probability_single_split)[::-1][:new_top])
    sorted_probability_split = np.array(probability_single_split)[sorted_index]
    new_train_label_single_split = new_train_label_single_split[sorted_index]
    filted_probability_split = sorted_probability_split[sorted_probability_split > probability_threshold]
    candidate_label = np.array(new_train_label_single_split[0])
    if 0 != len(filted_probability_split):
        candidate_label = new_train_label_single_split[:len(filted_probability_split)]
        #new_candidate_list.append(candidate_label)
    new_candidate_list[counter] = candidate_label
    counter += 1
    

CPU times: user 18 s, sys: 160 ms, total: 18.2 s
Wall time: 18.3 s


In [36]:
new_top_predict_label_list = np.zeros((len(new_candidate_list), 1))
for i in range(0, len(new_candidate_list)):
    predict_label = np.argmax(np.bincount(new_candidate_list[i]))
    new_top_predict_label_list[i] = predict_label

In [39]:
output_path = "../data/split_test/test_predict_label"
np.save(output_path, new_top_predict_label_list)